In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from keras.layers import Dense,InputLayer

In [5]:
class PINN(tf.keras.Model):
    def __init__(self):
        super (PINN,self).__init__()
        self.dense1 = Dense(100,activation='relu',inpu_dim=2)
        self.dense2 = Dense(100,activation='relu')
        self.output_layer = Dense(units=1,activation='sigmoid')

    def call(self,inputs):
        X = input[:,0:1]
        t = input[:,0:2]
        concat_inputs = tf.concat([X,t],axis=1)
        hidden_layer1 = self.dense1(concat_inputs)
        hidden_layer2 = self.dense2(hidden_layer1)
        output_layer1 = self.output_layer(hidden_layer2)

        return output_layer1

In [ ]:
def physics_loss_function(model,X,t):
    